## MSDS 432 - Programming Assignment #9 Dan Avni

The purpose of this programming assignment is 

In [26]:
#import required packages
import numpy as np
import pandas as pd

In [31]:
#adapted from: https://www.sanfoundry.com/dynamic-programming-solutions-edit-distance-problem/
def min_edit_distance(src, dest):
    len1, len2 = len(src), len(dest)
 
    #create a matrix of order (len1+1)*(len2+1) to memoize values
    edit = np.empty([len1+1,len2+1], dtype=int)
 
    #edit[i][j]=minimum number of edit operations required to transform src[0....(i-1)] to dest[0...(j-1)]
 
    for i in range(0, len1+1):
        edit[i][0]=i    #min operations required to transform src[0...i-1] to empty dest string
 
    for j in range(0, len2+1):
        edit[0][j]=j   #min operations required to transform empty src to dest[0...j-1]
 
    #now, start filling the matrix row wise
    for i in range(1, len1+1):
        for j in range(1, len2+1):
            #if current character of both strings match
            if(src[i-1]==dest[j-1]):            
                edit[i][j]=edit[i-1][j-1]
            #mismatch
            else:
                #try applying all operations and choose the one which costs minimum
                x=1+edit[i-1][j];    #delete 
                y=1+edit[i][j-1];    #insert
                z=1+edit[i-1][j-1];  #replace
                edit[i][j]=min(x,min(y,z))
                
    #now, return the final value 
    return edit[len1][len2];

In [32]:
min_edit_distance("danny", "mika")

5

In [33]:
min_edit_distance("mika", "danny")

5